<a href="https://colab.research.google.com/github/Leila828/instadeep_labs/blob/master/UNnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip -q install pydot_ng > /dev/null 2>&1
!pip -q install graphviz > /dev/null 2>&1
!apt install graphviz > /dev/null 2>&1

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D, UpSampling2D ,concatenate
from IPython import display
%matplotlib inline

In [ ]:
class MiniVGGNetModel(tf.keras.Model):
	def __init__(self, classes, chanDim=-1):
		# call the parent constructor
		super(MiniVGGNetModel, self).__init__()
		# initialize the layers in the first (CONV => RELU) * 2 => POOL
		# layer set
		self.conv1A = Conv2D(32, (3, 3), padding="same")
		self.act1A = Activation("relu")
		self.bn1A = BatchNormalization(axis=chanDim)
		self.conv1B = Conv2D(32, (3, 3), padding="same")
		self.act1B = Activation("relu")
		self.bn1B = BatchNormalization(axis=chanDim)
		self.pool1 = MaxPooling2D(pool_size=(2, 2))
		# initialize the layers in the second (CONV => RELU) * 2 => POOL
		# layer set
		self.conv2A = Conv2D(32, (3, 3), padding="same")
		self.act2A = Activation("relu")
		self.bn2A = BatchNormalization(axis=chanDim)
		self.conv2B = Conv2D(32, (3, 3), padding="same")
		self.act2B = Activation("relu")
		self.bn2B = BatchNormalization(axis=chanDim)
		self.pool2 = MaxPooling2D(pool_size=(2, 2))
		# initialize the layers in our fully-connected layer set
		self.flatten = Flatten()
		self.dense3 = Dense(512)
		self.act3 = Activation("relu")
		self.bn3 = BatchNormalization()
		self.do3 = Dropout(0.5)
		# initialize the layers in the softmax classifier layer set
		self.dense4 = Dense(classes)
		self.softmax = Activation("softmax")
  
  	def call(self, inputs):
		# build the first (CONV => RELU) * 2 => POOL layer set
		x = self.conv1A(inputs)
		x = self.act1A(x)
		x = self.bn1A(x)
		x = self.conv1B(x)
		x = self.act1B(x)
		x = self.bn1B(x)
		x = self.pool1(x)
		# build the second (CONV => RELU) * 2 => POOL layer set
		x = self.conv2A(x)
		x = self.act2A(x)
		x = self.bn2A(x)
		x = self.conv2B(x)
		x = self.act2B(x)
		x = self.bn2B(x)
		x = self.pool2(x)
		# build our FC layer set
		x = self.flatten(x)
		x = self.dense3(x)
		x = self.act3(x)
		x = self.bn3(x)
		x = self.do3(x)
		# build the softmax classifier
		x = self.dense4(x)
		x = self.softmax(x)
		# return the constructed model
		return x

In [ ]:
class ResnetBlock(Layer):
    def __init__(self, kernel_size, filters):
        super(ResnetBlock, self).__init__(name='')
        
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size)
        self.bn1 = BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size)
        self.bn2 = BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = tf.nn.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x += input_tensor
        return tf.nn.relu(x)

In [ ]:
class Unet(tf.keras.layers.Layer):
    def __init__(self, channel_start, nb_levels,classes):
        super(Unet, self).__init__(name='')
        
        self.channel_start =channel_start
        self.nb_levels=nb_levels
        self.classes=classes
        #level1
        self.res_block_1=ResnetBlock(3,112)
        self.maxpool_1=MaxPool2D()


        self.res_block_2=ResnetBlock(3,112)
        self.maxpool_2=MaxPool2D()



        self.res_block_3=ResnetBlock(3,112)
        self.maxpool_3=MaxPool2D()

        
        self.res_block_4=ResnetBlock(3,112)
        self.upsample_4=UpSampling2D()
        #decoder
 
        self.res_block_5=ResnetBlock(3,112)
        self.upsample_5=UpSampling2D()


         self.res_block_6=ResnetBlock(3,112)
        self.upsample_6=UpSampling2D()

       self.res_block_6=ResnetBlock(3,112)
       self.upsample_6=UpSampling2D()

       self.conv1(1,kernel, )

    def call(self, input, training=False):
      out_1=self.res_block_1(input)
      out_2=self.maxpool_1(out_1)
      out_3=self.res_block_2(out_2)
      out_4=self.maxpool_2(out_3)

      out_5=self.res_block_3(out_4)
      out_6=self.maxpool_3(out_5)


      out_7=self.res_block_4(out_6)
      out_8=self.upsample_4(out_7)

      out_9=self.res_block_5(concacatenate(out_5, out_8))
      out_10=self_upsample_5(out_9)

      out_11=self.res_block_6(concatenate(out_3, out_10))
      out_12=self_upsample_6(out_10)

      out_13=self.res_block_7(concatenate(out_1, out_12))
      out_14=self.conv1(out_13)

        return tf.nn.sigmoid()